## Accessing SQL via Python
In this continuation of the non-function notebook, we show how to wrap the DB functions in a library to create a more scalable way to use our code. 

We start by reading in the file as before. Note the import of our `wedge_helper` library, which is stored in a file with that same name and the extension `py`. Open that file up (maybe in Spyder?) so that you can see what's inside.

In [ ]:
import sqlite3
from wedge_helper import *

input_file = "OwnerTransactions_30.txt"

# Let's just open the file and read the first few lines to the screen.
with open(input_file,'r',encoding="Latin-1") as ifile :
    for idx, line in enumerate(ifile.readlines()) :
        print(line.strip().split("\t"))
        if idx > 3 :
            break

Let's do our standard open the DB in memory and create a cursor.

In [ ]:
db = sqlite3.connect("change_me.db") #':memory:') # Make this a directory + file if you want to store the results.
cur = db.cursor()

init_db(cur) # take a look at the .py file to see what happened here.

In [ ]:
with open(input_file,'r') as ifile :
    populate_db(db,ifile,delimiter="\t",limit=None)

And now let's run a query and print out the results in a semi-pretty fashion. Check out [this page](https://docs.python.org/3.2/library/string.html#format-specification-mini-language) to learn more about the formatting tricks.

In [ ]:
result = cur.execute('''SELECT card_no,
                               date(datetime) as date,
                               sum(total) AS spend
                        FROM transactions
                        WHERE trans_type = "I"
                        GROUP BY card_no, date
                        ''')

for idx,row in enumerate(result) :
    print("On {1}, card_no = {0} spent {2:,.2f}.".format(row[0],row[1],row[2]))
    if idx > 20 :
        break

Now let's return to our big query from the other notebook.

In [ ]:
result = cur.execute('''SELECT card_no,
                                   department,
                                   substr(date(datetime),1,4) AS year,
                                   substr(date(datetime),6,2) AS month,
                                   sum(total) AS spend,
                                   count(distinct(date(datetime) || register_no ||
                                           emp_no || trans_no)) as Transactions,
                                   sum(CASE WHEN (trans_status = 'V' or trans_status = 'R') THEN -1 ELSE 1 END) as Items
                                   FROM transactions
                                   WHERE department != 0 and
                                        department != 15 and
                                        trans_status != 'M' and
                                        trans_status != 'C' and
                                        trans_status != 'J' and
                                       (trans_status = '' or 
                                        trans_status = ' ' or 
                                        trans_status = 'V' or 
                                        trans_status = 'R') and card_no = 18736
                          GROUP BY card_no, department, year, month
                          ORDER BY year, month''')

In [ ]:
for row in result :
    print(row)

In [ ]:
for idx,row in enumerate(result) :
    print(row)
    if idx > 10 :
        break

Run the cell below as a best practice, but the DB will close if you quit the notebook. If it's an in-memory DB then it's also gone!

In [ ]:
db.close()